In [14]:
import sys
print(sys.executable, sys.version)


import numpy as np
import pandas as pd
import tensorflow as tf
import trimesh

from os.path import join, abspath
from IPython.display import display, HTML
from glob import glob

# for local import 
sys.path.append(abspath('../src/'))

from main.config import Config
from main.model import Model
from main.dataset import Dataset
from main.smpl import Smpl

from utils import draw_skeleton

/home/mmc-user/.virtualenvs/hmr2.0/bin/python3 3.6.8 (default, Oct  7 2019, 12:59:55) 
[GCC 8.3.0]


In [4]:
class EvalConfig(Config):
    ENCODER_ONLY = True
    LOG_DIR = join('/', 'data', 'ssd1', 'russales', 'logs', '22032020-231404')
    DATA_DIR = join('/', 'data', 'ssd1', 'russales', 'new_records')

# class Config is implemented as singleton, inizialize subclass first!
config = EvalConfig()

In [5]:
# inizialize model 
model = Model()

Saving logs to /data/ssd1/russales/logs/22032020-231404

Configurations:
BATCH_SIZE                     64
DATASETS                       ['lsp', 'lsp_ext', 'mpii', 'coco', 'mpii_3d', 'h36m']
DATA_DIR                       /data/ssd1/russales/new_records
DISCRIMINATOR_LEARNING_RATE    0.0001
DISCRIMINATOR_LOSS_WEIGHT      1
DISCRIMINATOR_WEIGHT_DECAY     0.0001
ENCODER_INPUT_SHAPE            (224, 224, 3)
ENCODER_LEARNING_RATE          1e-05
ENCODER_LOSS_WEIGHT            60.0
ENCODER_ONLY                   True
ENCODER_WEIGHT_DECAY           0.0001
EPOCHS                         55
ITERATIONS                     3
JOINT_TYPE                     cocoplus
LOG_DIR                        /data/ssd1/russales/logs/22032020-231404
NUM_CAMERA_PARAMS              3
NUM_JOINTS                     23
NUM_JOINTS_GLOBAL              24
NUM_KP2D                       19
NUM_KP3D                       14
NUM_PARALLEL                   16
NUM_POSE_PARAMS                72
NUM_SAMPLES                 

### run evaluation

In [ ]:
result = model.test(vis='True')

### sort results by sequence 

In [ ]:
all_kp3d_mpjpe = result['kp3d_mpjpe']
all_kp3d_mpjpe_aligned = result['kp3d_mpjpe_aligned']
sequences = result['sequences']

eval_dict = {}
indices = range(len(sequences))
for i, sequence in zip(indices, sequences):
    sequence = sequence.numpy().decode("utf-8")
    mpjpe_tuple = tuple([all_kp3d_mpjpe[i].numpy(), all_kp3d_mpjpe_aligned[i].numpy()])
    if sequence not in eval_dict.keys():
        eval_dict[sequence] = [mpjpe_tuple]
    else:
        eval_dict[sequence].append(mpjpe_tuple)

#### evaluate

In [ ]:
sequences, data = eval_dict.keys(), np.ndarray(shape=(len(eval_dict), 3), dtype=float)

indices = range(len(eval_dict))
for i, (_, v) in zip(indices, eval_dict.items()):
    data[i, 0] = np.mean(v[0])  # mean_error
    data[i, 1] = np.median(v[0])  # median_error
    data[i, 2] = np.mean(v[1])  # mean_error_aligned
        
columns = ['Mean', 'Median', 'Mean Aligned']
df = pd.DataFrame(data, index=sequences, columns=columns)
display(HTML(df.to_html()))

In [ ]:
df.describe()

### Overall results

In [ ]:
df_percentiles = pd.DataFrame({'90th Percentiles': np.percentile(all_kp3d_mpjpe, 90),
                        '70th Percentiles': np.percentile(all_kp3d_mpjpe, 70),
                        '50th Percentiles': np.percentile(all_kp3d_mpjpe, 50),
                        '30th Percentiles': np.percentile(all_kp3d_mpjpe, 30),
                        '10th Percentiles': np.percentile(all_kp3d_mpjpe, 10),
                        }, [1])

df_percentiles

In [ ]:
mpjpe = np.mean(all_kp3d_mpjpe)
mpjpe_median = np.median(all_kp3d_mpjpe)

mpjpe_aligned = np.mean(all_kp3d_mpjpe_aligned)
mpjpe_median_aligned = np.median(all_kp3d_mpjpe_aligned)

In [ ]:
df_all = pd.DataFrame({'Mean': mpjpe,
                        'Mean aligned': mpjpe_aligned,
                        'Median': mpjpe_median,
                        'Median aligned': mpjpe_median_aligned,
                        }, [1])
df_all

### Visualize

In [ ]:
thetas = result['thetas']
vertices = result['vertices']
rotations = result['rotations']

smpl = Smpl()

In [ ]:
mesh = trimesh.Trimesh(vertices=vertices[0], faces=smpl.get_faces(), process=False)

mesh.show()

### run inference and visualize

In [6]:
# load records
dataset = Dataset()
dataset = dataset.get_test()

initialize test dataset...
Done (t=0.30817508697509766)



In [15]:
for example in dataset.take(1):
    ground_truth = example
    result = model.detect(example[0])

In [16]:
print(ground_truth)

(<tf.Tensor: shape=(64, 224, 224, 3), dtype=float32, numpy=
array([[[[-0.4352941 , -0.654902  , -0.6862745 ],
         [-0.44313723, -0.6627451 , -0.69411767],
         [-0.42745095, -0.64705884, -0.6784314 ],
         ...,
         [-0.42745095, -0.61568624, -0.64705884],
         [-0.38823527, -0.57647055, -0.5921568 ],
         [-0.3960784 , -0.5607843 , -0.5686274 ]],

        [[-0.41176468, -0.6313726 , -0.6627451 ],
         [-0.41176468, -0.6313726 , -0.6627451 ],
         [-0.41176468, -0.6313726 , -0.6627451 ],
         ...,
         [-0.35686272, -0.545098  , -0.57647055],
         [-0.3098039 , -0.4980392 , -0.51372546],
         [-0.30196077, -0.49019605, -0.5058823 ]],

        [[-0.32549018, -0.55294114, -0.5843137 ],
         [-0.3333333 , -0.5607843 , -0.5921568 ],
         [-0.36470586, -0.5921568 , -0.6235294 ],
         ...,
         [-0.2862745 , -0.4823529 , -0.5215686 ],
         [-0.3098039 , -0.5058823 , -0.53725487],
         [-0.32549018, -0.51372546, -0.54509

In [13]:
original = ground_truth[0]
image = original[0].numpy()
kp2d = original[1].numpy()
image = draw_skeleton(, original[0][]kp2d, vis=vis)
plt.imshow(image)
plt.show()


(<tf.Tensor: shape=(64, 224, 224, 3), dtype=float32, numpy=
array([[[[-0.4352941 , -0.654902  , -0.6862745 ],
         [-0.44313723, -0.6627451 , -0.69411767],
         [-0.42745095, -0.64705884, -0.6784314 ],
         ...,
         [-0.42745095, -0.61568624, -0.64705884],
         [-0.38823527, -0.57647055, -0.5921568 ],
         [-0.3960784 , -0.5607843 , -0.5686274 ]],

        [[-0.41176468, -0.6313726 , -0.6627451 ],
         [-0.41176468, -0.6313726 , -0.6627451 ],
         [-0.41176468, -0.6313726 , -0.6627451 ],
         ...,
         [-0.35686272, -0.545098  , -0.57647055],
         [-0.3098039 , -0.4980392 , -0.51372546],
         [-0.30196077, -0.49019605, -0.5058823 ]],

        [[-0.32549018, -0.55294114, -0.5843137 ],
         [-0.3333333 , -0.5607843 , -0.5921568 ],
         [-0.36470586, -0.5921568 , -0.6235294 ],
         ...,
         [-0.2862745 , -0.4823529 , -0.5215686 ],
         [-0.3098039 , -0.5058823 , -0.53725487],
         [-0.32549018, -0.51372546, -0.54509